## 1. Upload data to S3
Here I use pokeman dataset as an example, which is composed of 833 image-text pairs. To scale up, you can just process your data into the same format.

In [1]:
# !pip install sagemaker

In [2]:
import sagemaker
import boto3
import datetime
import json
import os

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

In [4]:
prefix_data = 'datasets/midea_data'

local_data_path = "../data/midea_data"
input_data = sagemaker_session.upload_data(path=local_data_path, key_prefix=prefix_data)
print(input_data)

s3://sagemaker-us-west-2-452145973879/datasets/midea_data


## 2. Upload pretrained models to S3

In [5]:
prefix_model = 'models/whisper-large-v3'

local_model_path = "/home/ec2-user/SageMaker/efs/Models/whisper-large-v3"
input_model = sagemaker_session.upload_data(path=local_model_path, key_prefix=prefix_model)
print(input_model)

s3://sagemaker-us-west-2-452145973879/models/whisper-large-v3


## 3. Start a training job

In [11]:
import time
from sagemaker.estimator import Estimator

region = sagemaker_session.boto_session.region_name

image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.2.0-gpu-py310-cu121-ubuntu20.04-sagemaker'

instance_count = 1
# instance_type = 'ml.g5.48xlarge'
instance_type = 'ml.p4d.24xlarge' # 8 x A100 40G

checkpoint_local_path="/opt/ml/checkpoints"

environment = {
    'NODE_NUMBER': str(instance_count),
    'TRAIN_DATA_PATH': f's3://{bucket}/{prefix_data}/',
    'VALID_DATA_PATH': f's3://{bucket}/{prefix_data}/',
    'PRETRAINED_MODEL_S3_PATH': f"{input_model}/",
    'OUTPUT_MODEL_S3_PATH': f's3://{bucket}/checkpoints/whisper_checkpoint', # destination
}

estimator = Estimator(role=role,
                      entry_point='entry.py',
                      source_dir='./sm_scripts',
                      base_job_name='whisper-launch',
                      instance_count=instance_count,
                      instance_type=instance_type,
                      volume_size=1024, # in GB
                      image_uri=image_uri,
                      environment=environment,
                      max_run=3*24*3600, #任务最大存续时间，默认2day，需要提交ticket提升quota最大28天
                      disable_profiler=True,
                      debugger_hook_config=False,
                      checkpoint_s3_bucket=f's3://{bucket}/checkpoints/whisper_checkpoint')

estimator.fit()

INFO:sagemaker:Creating training-job with name: whisper-launch-2024-06-06-15-30-50-374


2024-06-06 15:30:52 Starting - Starting the training job...
2024-06-06 15:30:53 Pending - Training job waiting for capacity....................................................................................................................................................................................
2024-06-06 16:01:07 Pending - Preparing the instances for training........................
2024-06-06 16:05:18 Downloading - Downloading input data...
2024-06-06 16:05:48 Downloading - Downloading the training image.........
2024-06-06 16:07:34 Training - Training image download completed. Training in progress..........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-06-06 16:08:51,363 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-06-06 16:08:51,469 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-06-06 16:08:51,478 sagemaker_